# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 5 2022 8:47AM,247917,10,MSP214617,"Methapharm, Inc.",Released
1,Oct 5 2022 8:47AM,247917,10,MSP214619,"Methapharm, Inc.",Released
2,Oct 5 2022 8:47AM,247917,10,MSP214621,"Methapharm, Inc.",Released
3,Oct 5 2022 8:47AM,247917,10,MSP214620,"Methapharm, Inc.",Released
4,Oct 5 2022 8:47AM,247917,10,MSP214622,"Methapharm, Inc.",Released
5,Oct 5 2022 8:47AM,247917,10,MSP214623,"Methapharm, Inc.",Released
6,Oct 5 2022 8:47AM,247917,10,MSP214628,"Methapharm, Inc.",Released
7,Oct 5 2022 8:45AM,247915,10,0086050140,ISDIN Corporation,Released
8,Oct 5 2022 8:45AM,247915,10,0086050166,ISDIN Corporation,Released
9,Oct 5 2022 8:45AM,247915,10,0086050167,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,247913,Released,1
30,247914,Released,12
31,247915,Released,45
32,247916,Released,19
33,247917,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247913,NaN,NaN,1.0
247914,NaN,NaN,12.0
247915,NaN,NaN,45.0
247916,NaN,NaN,19.0
247917,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247811,24.0,6.0,0.0
247839,0.0,1.0,0.0
247841,12.0,0.0,49.0
247864,2.0,10.0,7.0
247875,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247811,24,6,0
247839,0,1,0
247841,12,0,49
247864,2,10,7
247875,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247811,24,6,0
1,247839,0,1,0
2,247841,12,0,49
3,247864,2,10,7
4,247875,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247811,24,6,
1,247839,,1,
2,247841,12,,49
3,247864,2,10,7
4,247875,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 5 2022 8:47AM,247917,10,"Methapharm, Inc."
7,Oct 5 2022 8:45AM,247915,10,ISDIN Corporation
52,Oct 5 2022 8:43AM,247916,10,ISDIN Corporation
71,Oct 5 2022 8:32AM,247914,10,ISDIN Corporation
83,Oct 5 2022 8:30AM,247900,10,ISDIN Corporation
155,Oct 5 2022 8:30AM,247905,10,"Nextsource Biotechnology, LLC"
156,Oct 5 2022 8:09AM,247913,10,"Senseonics, Incorporated"
157,Oct 5 2022 8:08AM,247912,10,Emerginnova
167,Oct 5 2022 7:36AM,247910,19,"AdvaGen Pharma, Ltd"
170,Oct 4 2022 5:10PM,247906,16,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 5 2022 8:47AM,247917,10,"Methapharm, Inc.",,,7
1,Oct 5 2022 8:45AM,247915,10,ISDIN Corporation,,,45
2,Oct 5 2022 8:43AM,247916,10,ISDIN Corporation,,,19
3,Oct 5 2022 8:32AM,247914,10,ISDIN Corporation,,,12
4,Oct 5 2022 8:30AM,247900,10,ISDIN Corporation,,,72
5,Oct 5 2022 8:30AM,247905,10,"Nextsource Biotechnology, LLC",,,1
6,Oct 5 2022 8:09AM,247913,10,"Senseonics, Incorporated",,,1
7,Oct 5 2022 8:08AM,247912,10,Emerginnova,,,10
8,Oct 5 2022 7:36AM,247910,19,"AdvaGen Pharma, Ltd",,,3
9,Oct 4 2022 5:10PM,247906,16,"Emersa Waterbox, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 8:47AM,247917,10,"Methapharm, Inc.",7,,
1,Oct 5 2022 8:45AM,247915,10,ISDIN Corporation,45,,
2,Oct 5 2022 8:43AM,247916,10,ISDIN Corporation,19,,
3,Oct 5 2022 8:32AM,247914,10,ISDIN Corporation,12,,
4,Oct 5 2022 8:30AM,247900,10,ISDIN Corporation,72,,
5,Oct 5 2022 8:30AM,247905,10,"Nextsource Biotechnology, LLC",1,,
6,Oct 5 2022 8:09AM,247913,10,"Senseonics, Incorporated",1,,
7,Oct 5 2022 8:08AM,247912,10,Emerginnova,10,,
8,Oct 5 2022 7:36AM,247910,19,"AdvaGen Pharma, Ltd",3,,
9,Oct 4 2022 5:10PM,247906,16,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 8:47AM,247917,10,"Methapharm, Inc.",7,,
1,Oct 5 2022 8:45AM,247915,10,ISDIN Corporation,45,,
2,Oct 5 2022 8:43AM,247916,10,ISDIN Corporation,19,,
3,Oct 5 2022 8:32AM,247914,10,ISDIN Corporation,12,,
4,Oct 5 2022 8:30AM,247900,10,ISDIN Corporation,72,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 8:47AM,247917,10,"Methapharm, Inc.",7.0,NaN,NaN
1,Oct 5 2022 8:45AM,247915,10,ISDIN Corporation,45.0,NaN,NaN
2,Oct 5 2022 8:43AM,247916,10,ISDIN Corporation,19.0,NaN,NaN
3,Oct 5 2022 8:32AM,247914,10,ISDIN Corporation,12.0,NaN,NaN
4,Oct 5 2022 8:30AM,247900,10,ISDIN Corporation,72.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 5 2022 8:47AM,247917,10,"Methapharm, Inc.",7.0,0.0,0.0
1,Oct 5 2022 8:45AM,247915,10,ISDIN Corporation,45.0,0.0,0.0
2,Oct 5 2022 8:43AM,247916,10,ISDIN Corporation,19.0,0.0,0.0
3,Oct 5 2022 8:32AM,247914,10,ISDIN Corporation,12.0,0.0,0.0
4,Oct 5 2022 8:30AM,247900,10,ISDIN Corporation,72.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3222639,177.0,16.0,26.0
15,495737,50.0,0.0,25.0
16,991569,18.0,0.0,0.0
18,495723,0.0,2.0,0.0
19,991563,16.0,5.0,11.0
20,247875,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3222639,177.0,16.0,26.0
1,15,495737,50.0,0.0,25.0
2,16,991569,18.0,0.0,0.0
3,18,495723,0.0,2.0,0.0
4,19,991563,16.0,5.0,11.0
5,20,247875,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,177.0,16.0,26.0
1,15,50.0,0.0,25.0
2,16,18.0,0.0,0.0
3,18,0.0,2.0,0.0
4,19,16.0,5.0,11.0
5,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,177.0
1,15,Released,50.0
2,16,Released,18.0
3,18,Released,0.0
4,19,Released,16.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20
Status,,,,,,
Completed,26.0,25.0,0.0,0.0,11.0,0.0
Executing,16.0,0.0,0.0,2.0,5.0,0.0
Released,177.0,50.0,18.0,0.0,16.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20
0,Completed,26.0,25.0,0.0,0.0,11.0,0.0
1,Executing,16.0,0.0,0.0,2.0,5.0,0.0
2,Released,177.0,50.0,18.0,0.0,16.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20
0,Completed,26.0,25.0,0.0,0.0,11.0,0.0
1,Executing,16.0,0.0,0.0,2.0,5.0,0.0
2,Released,177.0,50.0,18.0,0.0,16.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()